In [1]:
import sys
import os
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd

# Avoids warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
import tensorflow as tf

import ScalableLib.classifier.Multiband as multiband

2024-08-27 09:00:56.101825: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-27 09:00:56.101862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-27 09:00:56.102842: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-27 09:00:57.367147: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# To see if the system recognises the GPU
device = 1
devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(devices[device], 'GPU')
tf.config.experimental.set_memory_growth(device=devices[device], enable=True)

device_name = tf.config.experimental.get_device_details(devices[device])['device_name']
print("Using {}".format(device_name))

Using NVIDIA GeForce RTX 2080 Ti


Find the different folds and train a model using the stored data.

In [3]:
survey = 'ZTF'
path = os.path.join('../../02_CreateRecords/', survey, 'Folds/Fold_*',)
folds = glob(path)
folds.sort()
folds

['../../02_CreateRecords/ZTF/Folds/Fold_1',
 '../../02_CreateRecords/ZTF/Folds/Fold_2',
 '../../02_CreateRecords/ZTF/Folds/Fold_3',
 '../../02_CreateRecords/ZTF/Folds/Fold_4',
 '../../02_CreateRecords/ZTF/Folds/Fold_5',
 '../../02_CreateRecords/ZTF/Folds/Fold_6',
 '../../02_CreateRecords/ZTF/Folds/Fold_7']

Create folder results

In [4]:
if not os.path.exists('./Results'):
    os.mkdir('./Results')


Define the arguments for all the models.

In [5]:
train_args = {
            'hidden_size_bands':[128,128, 128],
            'hidden_size_central':[128, 128],
            'fc_layers_bands':[128,128,128],
            'fc_layers_central':[128,128,128], # Neurons of each layer
            'regression_size':[128, 128],#each element is a layer with that size.
            'buffer_size':10000,
            'epochs':1000,
            'num_threads':7,
            'batch_size':128,
            'dropout':0.30,
            'lr':[[5e-3]*2, 2.5e-3], # [[band1, band2], central]
            'val_steps':50,
            'max_to_keep':0, # Not Used 
            'steps_wait':500, 
            'use_class_weights':True,# Not Used as intended, for initialization
            'mode' : 'classifier'
            }
loss_weights = {'Class':1.0}

callbacks_args = {'patience': 20,
                  'mode':'max',
                  'restore_best_weights':True,
                  'min_delta': 0.001
                 }
train_args_specific={
                    'phys_params': [],
                    'use_output_bands' : True,  # Working
                    'use_output_central' : False, # Not used
                    'use_common_layers' : False, # NOT Working
                    'bidirectional_central' : False,# Working
                    'bidirectional_band' : False,# Not Working
                    'layer_norm_params' : None, # Used to normalyze common layers
                    'use_gated_common' : False, # Working
                    'l1':0.0,
                    'l2':0.0,  
                    'N_skip': 3, # Cannot be greater than the number of timesteps
                    'use_raw_input_central': False,
                    'train_steps_central' : 2,
                    'print_report' : True,
                    'loss_weights_central' : loss_weights,
                    'callbacks_args':callbacks_args
                    }



In [6]:
for fold in folds:
    tf.keras.backend.clear_session()
    # Set the fold path
    base_dir = fold+'/'
    
    # Set the save path for this fold. Create folder if needed
    path_results_fold = fold.replace('../../02_CreateRecords/'+survey+'/', './').replace('/Folds/', '/Results/')
    if not os.path.exists(path_results_fold):
        os.mkdir(path_results_fold)
    
    train_args_specific['save_dir'] = path_results_fold
    train_args_specific['metadata_pre_path'] = base_dir+'metadata_preprocess.json'
    train_args_specific['path_scalers'] =  os.path.join(fold,'scalers.pkl')
                                 
    # Define the train args
    train_args = {**train_args, **train_args_specific}


    train_files = base_dir+'train/*.tfrecord'
    val_files = base_dir+'val/*.tfrecord'
    test_files = base_dir+'test/*.tfrecord' 
    
    new = multiband.Network()
    new.train(train_args, train_files, val_files, test_files)    
    new.train_loop()

./Results/Fold_1/Models/20240827-0901
Start training


I0000 00:00:1724763730.011547  338782 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


41/41 [==============================] - 16s 384ms/step - loss: 48.3538 - Class_loss: 48.3538 - Class_CentralAcc: 0.9173 - Class_CentralTop2: 0.9804 - Class_FinalAcc: 0.9447 - Class_FinalTop2: 0.9890 - Class_Final_FScore: 0.8347
Early Stopping
              precision    recall  f1-score   support

         AGN       0.66      0.60      0.63       484
      Blazar       0.70      0.59      0.64       218
     CV/Nova       0.85      0.83      0.84       152
           E       0.92      0.95      0.94      2000
         LPV       0.99      1.00      0.99      1863
         QSO       0.89      0.92      0.90      2000
         RRL       0.94      0.93      0.94      2000
        SNIa       1.00      0.92      0.96       101
         YSO       0.84      0.79      0.81       159

    accuracy                           0.91      8977
   macro avg       0.87      0.84      0.85      8977
weighted avg       0.91      0.91      0.91      8977

./Results/Fold_2/Models/20240827-1843
Start trainin

Exception ignored in: <function _TFShouldUseHelper.__del__ at 0x7ff30501c280>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/tf_should_use.py", line 88, in __del__
    def __del__(self):
KeyboardInterrupt: 

KeyboardInterrupt

